# 文本内容补全初探（Completions API）[Legacy]

使用 Completions API 实现各类文本生成任务，主要请求参数说明：
- **`model`** （string，必填）：要使用的模型的 ID。可以参考 **模型端点兼容性表**。
- **`prompt`** （string or array，必填，Defaults to ）：生成补全的提示，编码为字符串、字符串数组、token数组或token数组数组。注意，这是模型在训练过程中看到的文档分隔符，所以如果没有指定提示符，模型将像从新文档的开头一样生成。
- **`stream`** （boolean，选填，默认 false）：当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容，即会不断地输出内容直到完成响应，流通过 `data: [DONE]` 消息终止。
- **`max_tokens`** （integer，选填，默认是 16）：补全时要生成的最大 token 数。提示 `max_tokens` 的 token 计数不能超过模型的上下文长度。大多数模型的上下文长度为 2048 个token（最新模型除外，它支持 4096）
- **`temperature`** （number，选填，默认是1）：使用哪个采样温度，在 **0和2之间**。较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。通常建议修改这个（`temperature` ）或 `top_p` 但两者不能同时存在，二选一。
- **`n`** （integer，选填，默认为 1）：每个 `prompt` 生成的补全次数。注意：由于此参数会生成许多补全，因此它会快速消耗token配额。小心使用，并确保对 `max_tokens` 和 `stop` 进行合理的设置。

## 生成英文文本

In [2]:
import os
from openai import OpenAI

client = OpenAI()
data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

print(data)

Completion(id='cmpl-96Y4rGbW0PRZLChDBvHjqoTnsGh5p', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\nThis is a test.')], created=1711347989, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=5, total_tokens=11))


In [3]:
text = data.choices[0].text
print(text)



This is a test.


## 生成中文文本
调整 `max_tokens` 

In [4]:
data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="讲10个给程序员听得笑话",
  max_tokens=1000,
  temperature=0.5
)

text = data.choices[0].text
print(text)



1.程序员和他的女朋友在一起，她问：“你在想什么？”他回答：“我在想，如果我把你的名字作为变量，会不会让我的生活变得更有意义。”

2.为什么程序员喜欢用黑色背景？因为黑色背景可以节省电力，让他们的代码运行得更快。

3.为什么程序员总是喜欢喝茶？因为他们喜欢在代码中加入一些Java。

4.程序员的女朋友抱怨他从来不给她送花，他回答：“我可以给你一束花，但是它会有100个电子版本。”

5.为什么程序员总是喜欢用鼠标右键？因为他们总是想要做正确的事情。

6.程序员和非程序员的区别是什么？程序员在睡觉时也在做梦，但是他们的梦都是代码。

7.为什么程序员总是喜欢用Linux？因为他们喜欢挑战，就像是在玩一个永远无法通关的游戏。

8.为什么程序员总是喜欢用英文命名变量？因为他们不想让别人知道他们的代码是怎么写的。

9.程序员和魔术师的相似之处是什么？他们都能把一堆代码变成一个神奇的程序。

10.程序员的幽默感就像是一个Bug，有些人能理解，有些人永远无法理解。


## 生成 Python 代码，并执行和验证
以面试中考察的典型的试题 `快速排序` 为例

In [5]:
data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="生成可执行的快速排序 Python 代码",
  max_tokens=1000,
  temperature=0
)

text = data.choices[0].text
print(text)



def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    left = [x for x in arr[1:] if x <= pivot]
    right = [x for x in arr[1:] if x > pivot]
    return quick_sort(left) + [pivot] + quick_sort(right)


#### Prompt：Jupyter Notebook 中执行生成的代码

Prompt：

```
我现在用 Completion API 生成了 Python 代码，并以字符串形式存放在 text 中，如下所示：

text = data.choices[0].text
print(text)

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    left = [x for x in arr[1:] if x <= pivot]
    right = [x for x in arr[1:] if x > pivot]
    return quick_sort(left) + [pivot] + quick_sort(right)

如何在 Jupyter notebook 中执行text中存放的这段代码
```

In [6]:
# `exec` 函数会执行传入的字符串作为 Python 代码。
# 在这个例子中，我们使用 `exec` 来定义了一个 `quick_sort` 函数，然后你就可以调用这个函数了。
# 请注意，`exec` 可以执行任何 Python 代码，因此在使用它的时候一定要小心，特别是当你执行的代码来自不可信的来源时。
exec(text)

In [7]:
# 现在你可以调用这个函数了
print(quick_sort([12,3,6,8,10,1,2,1]))

[1, 1, 2, 3, 6, 8, 10, 12]


# 聊天机器人初探（Chat Completions API）

使用 Chat Completions API 实现对话任务

聊天补全(Chat Completions API)以消息列表作为输入，并返回模型生成的消息作为输出。尽管聊天格式旨在使多轮对话变得简单，但它同样适用于没有任何对话的单轮任务。

主要请求参数说明：


- **`model` （string，必填）**

  要使用的模型ID。有关哪些模型适用于Chat API的详细信息

- **`messages` （array，必填）**

  迄今为止描述对话的消息列表
    - **`role` （string，必填）**

  发送此消息的角色。`system` 、`user` 或 `assistant` 之一（一般用 user 发送用户问题，system 发送给模型提示信息）

    - **`content` （string，必填）**
    
      消息的内容
    
    - **`name` （string，选填）**
    
      此消息的发送者姓名。可以包含 a-z、A-Z、0-9 和下划线，最大长度为 64 个字符

- **`stream` （boolean，选填，是否按流的方式发送内容）**

  当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容。SSE 本质上是一个长链接，会持续不断地输出内容直到完成响应。如果不是做实时聊天，默认false即可。

- **`max_tokens` （integer，选填）**

  在聊天补全中生成的最大 **tokens** 数。

  输入token和生成的token的总长度受模型上下文长度的限制。

- **`temperature` （number，选填，默认是 1）**

  采样温度，在 0和 2 之间。

  较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。

  通常建议修改这个（`temperature` ）或者 `top_p` ，但两者不能同时存在，二选一。


## 开启聊天模式

使用 `messages` 记录迄今为止对话的消息列表

In [20]:
from openai import OpenAI
client = OpenAI()

messages=[
    {
        "role": "user", 
        "content": "Hello!"
    }
]


data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages = messages
)


In [21]:
print(data)

ChatCompletion(id='chatcmpl-8P2Orm70Z8O9pAz4fdM0JDJoHrvxY', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700978117, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))


In [22]:
# 从返回的数据中获取生成的消息
new_message = data.choices[0].message
# 打印 new_message
print(new_message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [23]:
# 将消息追加到 messages 列表中
messages.append(new_message)
print(messages)

[{'role': 'user', 'content': 'Hello!'}, ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)]


In [24]:
type(new_message)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [25]:
new_message.role

'assistant'

In [26]:
new_message.content

'Hello! How can I assist you today?'

In [27]:
messages.pop()

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)

In [28]:
print(messages)

[{'role': 'user', 'content': 'Hello!'}]


#### Prompt: OpenAIObject -> Dict

```
打印 messages 列表后发现数据类型不对，messages 输出如下：

print(messages)

[{'role': 'user', 'content': 'Hello!'}, <OpenAIObject at 0x7f27582c13f0> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}]

将OpenAIObject 转换为一个如下数据类型格式：

    {
        "role": "user", 
        "content": "Hello!"
    }
```

In [29]:
new_message = data.choices[0].message
new_message_dict = {"role": new_message.role, "content": new_message.content}
type(new_message_dict)

dict

In [30]:
print(new_message_dict)

{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}


In [31]:
# 将消息追加到 messages 列表中
messages.append(new_message_dict)

In [32]:
print(messages)

[{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]


#### 新一轮对话

In [33]:
new_chat = {
    "role": "user",
    "content": "1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？"
}

In [34]:
messages.append(new_chat)

In [35]:
from pprint import pprint

pprint(messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': 'Hello! How can I assist you today?', 'role': 'assistant'},
 {'content': '1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？', 'role': 'user'}]


In [36]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [37]:
new_message = data.choices[0].message
# 打印 new_messages 
print(new_message)

ChatCompletionMessage(content='1. 冷笑话：为什么程序员总是喜欢用Mac电脑？因为他们觉得Windows太常见，Linux太复杂，而Mac正好是个折中方案，里面融合了太多他们从未使用的技术！\n2. 今天日期是根据您所在的时区而定。可以告诉我你所在的时区吗？\n3. 明天是根据您所在的时区而定。可以告诉我你所在的时区吗？', role='assistant', function_call=None, tool_calls=None)


In [38]:
# 打印 new_messages 内容
print(new_message.content)

1. 冷笑话：为什么程序员总是喜欢用Mac电脑？因为他们觉得Windows太常见，Linux太复杂，而Mac正好是个折中方案，里面融合了太多他们从未使用的技术！
2. 今天日期是根据您所在的时区而定。可以告诉我你所在的时区吗？
3. 明天是根据您所在的时区而定。可以告诉我你所在的时区吗？


## 使用多种身份聊天对话

目前`role`参数支持3类身份： `system`, `user` `assistant`:


![](images/chat_completion_api.png)



In [39]:
# 构造聊天记录
messages=[
    {"role": "system", "content": "你是一个乐于助人的体育界专家。"},
    {"role": "user", "content": "2008年奥运会是在哪里举行的？"},
]

In [40]:
import openai

data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)


In [41]:
message = data.choices[0].message.content
print(message)

2008年奥运会是在中国的北京市举行的。有什么其他的问题我可以回答吗？


In [42]:
# 添加 GPT 返回结果到聊天记录
messages.append({"role": "assistant", "content": message})

In [43]:
messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant', 'content': '2008年奥运会是在中国的北京市举行的。有什么其他的问题我可以回答吗？'}]

In [44]:
# 第二轮对话
messages.append({"role": "user", "content": "1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？"})

In [45]:
messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant', 'content': '2008年奥运会是在中国的北京市举行的。有什么其他的问题我可以回答吗？'},
 {'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]

In [46]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [47]:
message = data.choices[0].message.content
print(message)

1. 2008年奥运会中，金牌最多的国家是中国。中国取得了51枚金牌。
2. 奖牌最多的国家同样是中国。中国在2008年奥运会中获得了100枚奖牌，其中包括51枚金牌、21枚银牌和28枚铜牌。有没有其他的问题呢？


In [48]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]
)

In [49]:
data.choices[0].message.content

'1. 目前，美国是金牌最多的国家。根据2021年东京奥运会数据，美国的金牌总数为1137枚。\n2. 目前，美国是奖牌最多的国家。根据2021年东京奥运会数据，美国的奖牌总数为2820枚。'